<a href="https://colab.research.google.com/github/TomasVendel/ASESOR-FINANCIERO/blob/main/ASESOR_FINANCIERO_PYTHON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=598638171c0d7c00f31b29d54acffa2fae59d572d5283582c008a16cb4ae06ba
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [14]:
import numpy as np
np.NaN = np.nan


In [ ]:
import yfinance as yf
import pandas_ta as ta
import matplotlib.pyplot as plt
import pandas as pd

print(" -------💸📈 BIENVENIDO A TU ASESOR FINANCIERO PERSONAL EN PYTHON 📈💸------- ")

def obtener_perfil():
    """
    Solicita el perfil de inversor.
    Se muestran las opciones:
      1 - Conservador
      2 - Moderado
      3 - Agresivo
      0 - Volver
    Además, se muestra el mensaje: “Si no sabes qué tipo de inversor sos, podés hacer un test del Inversor acá: <URL>”
    Devuelve:
      - Un entero (1, 2 o 3) si la elección es válida.
      - None si se pulsa 0.
    """
    print("\n¿Qué tipo de inversor sos?")
    print("1 - Conservador")
    print("2 - Moderado")
    print("3 - Agresivo")
    print("0 - Volver")
    print("Si no sabes qué tipo de Inversor eres, podés hacer un TEST DEL INVERSOR aquí: https://iol.invertironline.com/User/test-del-inversor?_gl=1*1lhem7l*_gcl_au*MjAxNjkyMjc5NS4xNzQ4OTc3OTM3*_ga*MTY2NzgwMzY4OC4xNzQ4OTc3OTM3*_ga_3ZJ55WFL39*czE3NDg5Nzc5MzckbzEkZzEkdDE3NDg5NzgwMjUkajQxJGwwJGgxNjExNjAyNDY1")
    print("")
    while True:
        respuesta = input("Elegí una opción (0/1/2/3): ")
        if respuesta == "0":
            return None
        if respuesta in ['1','2','3']:
            return int(respuesta)
        print("Opción inválida.")

def obtener_horizonte():
    """
    Solicita el horizonte de inversión.
    Opción 0 indica volver (regresa al nivel anterior, es decir, a la selección del perfil).
    Devuelve:
      - Un entero (1, 2 o 3) si la elección es válida.
      - None si se pulsa 0.
    """
    print("\n¿Cuál es tu horizonte de inversión?")
    print("1 - Corto plazo (0-12 meses)")
    print("2 - Mediano plazo (1-3 años)")
    print("3 - Largo plazo (más de 3 años)")
    print("0 - Volver")
    while True:
        respuesta = input("Elegí una opción (0/1/2/3): ")
        if respuesta == "0":
            return None
        if respuesta in ['1','2','3']:
            return int(respuesta)
        print("Opción inválida.")

def recomendar_portafolio(perfil):
    """
    Retorna una distribución sugerida según el perfil del inversor.
    """
    if perfil == 1:
        return {'Renta fija': 60, 'Acciones': 20, 'ETFs': 10, 'Criptomonedas': 10}
    elif perfil == 2:
        return {'Renta fija': 30, 'Acciones': 40, 'ETFs': 20, 'Criptomonedas': 10}
    else:
        return {'Renta fija': 10, 'Acciones': 50, 'ETFs': 20, 'Criptomonedas': 20}

def elegir_distribucion_personalizada():
    """
    Permite al usuario ingresar una distribución personalizada.
    Los porcentajes deben sumar 100.
    En cada entrada se ofrece la opción "0" para volver (regresar a la pregunta de distribución).
    Si se ingresa 0 se retorna None.
    """
    print("\n🔧 Ingresá tu distribución personalizada (los porcentajes deben sumar 100):")
    tipos = ['Renta fija', 'Acciones', 'ETFs', 'Criptomonedas']
    distribucion = {}
    total = 0
    for tipo in tipos:
        while True:
            val = input(f"% a invertir en {tipo} (o SALIR para volver): ").upper()

            if val.strip() == "SALIR":
                return None
            try:
                porcentaje = float(val)
            except ValueError:
                print("Ingresá un número válido.")
                continue
            if 0 <= porcentaje <= 100:
                distribucion[tipo] = porcentaje
                total += porcentaje
                break
            else:
                print("Por favor, ingresá un valor entre 0 y 100.")
    if round(total, 2) != 100.0:
        print("\n❌ Los porcentajes no suman 100. Volvé a intentarlo.")
        return elegir_distribucion_personalizada()
    return distribucion

def mostrar_grafico(distribucion):
    """
    Muestra un gráfico de pastel con la distribución del portafolio.
    """
    labels = distribucion.keys()
    sizes = distribucion.values()
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors)
    plt.axis('equal')
    plt.title("Distribución del Portafolio")
    plt.show()

def mostrar_desglose(distribucion, capital):
    """
    Muestra el desglose del capital para cada tipo de inversión.
    """
    print("\n📊 Distribución seleccionada:")
    for tipo, porcentaje in distribucion.items():
        monto = capital * porcentaje / 100
        print(f"- {tipo}: ${monto:.2f} ({porcentaje}%)")

def analizar_acciones_rsi_macd(simbolos, horizonte):
    """
    Analiza cada acción usando RSI y MACD.
    La temporalidad se adapta según el horizonte:
      - Corto plazo: period="3mo", interval="1d"
      - Mediano plazo: period="1y", interval="1wk"
      - Largo plazo: period="5y", interval="1mo"
    """
    print("\n🔍 Analizando RSI y MACD de acciones...\n")
    if horizonte == 1:
        period, interval = "3mo", "1d"
    elif horizonte == 2:
        period, interval = "1y", "1wk"
    elif horizonte == 3:
        period, interval = "5y", "1mo"
    else:
        period, interval = "3mo", "1d"

    lower_threshold = 30
    upper_threshold = 70

    for ticker in simbolos:
        try:
            data = yf.download(ticker, period=period, interval=interval, progress=False)
            if isinstance(data.columns, pd.MultiIndex):
                if "Close" in data.columns.get_level_values(1):
                    data.columns = data.columns.get_level_values(1)
                else:
                    data.columns = ['_'.join(map(str, col)).strip('_') for col in data.columns.values]
            if data.empty:
                print(f"⚠️ {ticker}: Sin datos disponibles.")
                continue

            data.ta.rsi(length=14, append=True)
            if "RSI_14" not in data.columns:
                print(f"⚠️ {ticker}: RSI no disponible.")
                continue
            rsi = data["RSI_14"].iloc[-1]
            if rsi <= lower_threshold:
                rsi_estado = "\033[92mSobrevendida, posible compra\033[0m"
            elif rsi >= upper_threshold:
                rsi_estado = "\033[91mSobrecomprada, posible venta\033[0m"
            else:
                rsi_estado = "\033[93mNeutral\033[0m"
            print(f"{ticker}: RSI = {rsi:.2f} → {rsi_estado}")

            if "Close" not in data.columns:
                found = False
                for col in data.columns:
                    if "close" in col.lower():
                        data["Close"] = data[col]
                        found = True
                        break
                if not found:
                    print(f"⚠️ {ticker}: La columna 'Close' no se encontró.")
                    print("-" * 70)
                    continue

            macd = ta.macd(data["Close"])
            data = data.join(macd)
            req_cols = ["MACD_12_26_9", "MACDs_12_26_9", "MACDh_12_26_9"]
            last_macd = data[req_cols].iloc[-1]
            if last_macd.isna().any():
                print(f"⚠️ {ticker}: MACD no disponible.")
                print("-" * 70)
                continue
            macdh = last_macd["MACDh_12_26_9"]
            if macdh > 0:
                histo_status = "\033[92mMomentum alcista\033[0m"
            elif macdh < 0:
                histo_status = "\033[91mMomentum bajista\033[0m"
            else:
                histo_status = "\033[93mSin dirección clara\033[0m"
            print(f"{ticker}: MACD Histograma = {macdh:.2f} → {histo_status}")

            try:
                last2 = data[req_cols].iloc[-2]
            except IndexError:
                print(f"⚠️ {ticker}: No hay suficientes datos para evaluar cruces.")
                print("-" * 70)
                continue

            macd_curr = last_macd["MACD_12_26_9"]
            signal_curr = last_macd["MACDs_12_26_9"]
            macd_prev = last2["MACD_12_26_9"]
            signal_prev = last2["MACDs_12_26_9"]

            if macd_prev < signal_prev and macd_curr > signal_curr:
                print(f"📈 {ticker}: Cruce alcista detectado (compra)")
            elif macd_prev > signal_prev and macd_curr < signal_curr:
                print(f"📉 {ticker}: Cruce bajista detectado (venta)")

            print("-" * 70)
        except Exception as e:
            print(f"Error al analizar MACD de {ticker}: {e}")

def analizar_cripto(simbolos, horizonte):
    """
    Analiza cada criptomoneda usando RSI y MACD.
    La temporalidad depende del horizonte.
    """
    print("\n🔍 Analizando RSI y MACD de criptomonedas...\n")
    if horizonte == 1:
        period, interval = "3mo", "1d"
    elif horizonte == 2:
        period, interval = "1y", "1wk"
    elif horizonte == 3:
        period, interval = "5y", "1mo"
    else:
        period, interval = "3mo", "1d"
    lower_threshold = 30
    upper_threshold = 70

    for ticker in simbolos:
        try:
            data = yf.download(ticker, period=period, interval=interval, progress=False)
            if data.empty:
                print(f"⚠️ {ticker}: Sin datos disponibles.")
                continue
            if isinstance(data.columns, pd.MultiIndex):
                if "Close" in data.columns.get_level_values(1):
                    data.columns = data.columns.get_level_values(1)
                else:
                    data.columns = ['_'.join(map(str, col)).strip('_') for col in data.columns.values]
            data.ta.rsi(length=14, append=True)
            if "RSI_14" not in data.columns:
                print(f"⚠️ {ticker}: RSI no disponible.")
                continue
            rsi = data["RSI_14"].iloc[-1]
            if rsi <= lower_threshold:
                rsi_estado = "\033[92mSobrevendida, posible compra\033[0m"
            elif rsi >= upper_threshold:
                rsi_estado = "\033[91mSobrecomprada, posible venta\033[0m"
            else:
                rsi_estado = "\033[93mNeutral\033[0m"
            print(f"{ticker}: RSI = {rsi:.2f} → {rsi_estado}")
            if "Close" not in data.columns:
                found = False
                for col in data.columns:
                    if "close" in col.lower():
                        data["Close"] = data[col]
                        found = True
                        break
                if not found:
                    print(f"⚠️ {ticker}: La columna 'Close' no se encontró.")
                    print("-" * 70)
                    continue
            macd = ta.macd(data["Close"])
            data = data.join(macd)
            req_cols = ["MACD_12_26_9", "MACDs_12_26_9", "MACDh_12_26_9"]
            last_macd = data[req_cols].iloc[-1]
            if last_macd.isna().any():
                print(f"⚠️ {ticker}: MACD no disponible.")
                print("-" * 70)
                continue
            macdh = last_macd["MACDh_12_26_9"]
            if macdh > 0:
                histo_status = "\033[92mMomentum alcista\033[0m"
            elif macdh < 0:
                histo_status = "\033[91mMomentum bajista\033[0m"
            else:
                histo_status = "\033[93mSin dirección clara\033[0m"
            print(f"{ticker}: MACD Histograma = {macdh:.2f} → {histo_status}")
            try:
                last2 = data[req_cols].iloc[-2]
            except IndexError:
                print(f"⚠️ {ticker}: No hay suficientes datos para evaluar cruces.")
                print("-" * 70)
                continue
            macd_curr = last_macd["MACD_12_26_9"]
            signal_curr = last_macd["MACDs_12_26_9"]
            macd_prev = last2["MACD_12_26_9"]
            signal_prev = last2["MACDs_12_26_9"]
            if macd_prev < signal_prev and macd_curr > signal_curr:
                print(f"📈 {ticker}: Cruce alcista detectado (compra)")
            elif macd_prev > signal_prev and macd_curr < signal_curr:
                print(f"📉 {ticker}: Cruce bajista detectado (venta)")
            print("-" * 70)
        except Exception as e:
            print(f"Error al analizar MACD de {ticker}: {e}")

def recomendar_renta_fija(horizonte):
    """
    Recomienda productos de renta fija según el horizonte de inversión.

    Horizonte:
      1 → Corto plazo (0-12 meses):
          - US Treasury:
              • T-Bill a 3 meses: Rendimiento anual aprox. 2.5%
              • T-Bill a 6 meses: Rendimiento anual aprox. 2.8%
          - Argentina:
              • Letras del Tesoro: Rendimiento anual aprox. 10%-12%
              • Plazo Fijo en Banco Nación (30 días): TNA aprox. 45%-50%
      2 → Mediano plazo (1-3 años):
          - US Treasury:
              • Treasury Note a 2 años: Rendimiento anual aprox. 2.5%
              • Treasury Note a 5 años: Rendimiento anual aprox. 3%
          - Argentina:
              • Bonos soberanos (Ej: AL30 o GD30): Rendimiento anual aprox. 15%-20%
              • Plazo Fijo en Banco Galicia (90 días): TNA aprox. 55%-60%
      3 → Largo plazo (más de 3 años):
          - US Treasury:
              • Treasury Bond a 10 años: Rendimiento anual aprox. 2.5%
              • Treasury Bond a 20 años: Rendimiento anual aprox. 2.8%
              • Treasury Bond a 30 años: Rendimiento anual aprox. 3%
          - Argentina:
              • Bonos soberanos/corporativos (Ej: AL30): Rendimiento anual aprox. 20%-25%
              • Plazo Fijo en Banco Santander Río (180 días): TNA aprox. 60%-70%
    """
    print("\n🏦 Recomendaciones de Renta Fija:")
    if horizonte == 1:
        print("Para un horizonte de corto plazo (0-12 meses):")
        print(" US Treasury:")
        print("  - T-Bill a 3 meses: Rendimiento anual aprox. 2.5%")
        print("  - T-Bill a 6 meses: Rendimiento anual aprox. 2.8%")
        print(" Argentina:")
        print("  - Letras del Tesoro: Rendimiento anual aprox. 10%-12%")
        print("  - Plazo Fijo en Banco Nación (30 días): TNA aprox. 45%-50%")
    elif horizonte == 2:
        print("Para un horizonte de mediano plazo (1-3 años):")
        print(" US Treasury:")
        print("  - Treasury Note a 2 años: Rendimiento anual aprox. 2.5%")
        print("  - Treasury Note a 5 años: Rendimiento anual aprox. 3%")
        print(" Argentina:")
        print("  - Bonos soberanos (Ej: AL30 o GD30): Rendimiento anual aprox. 15%-20%")
        print("  - Plazo Fijo en Banco Galicia (90 días): TNA aprox. 55%-60%")
    elif horizonte == 3:
        print("Para un horizonte de largo plazo (más de 3 años):")
        print(" US Treasury:")
        print("  - Treasury Bond a 10 años: Rendimiento anual aprox. 2.5%")
        print("  - Treasury Bond a 20 años: Rendimiento anual aprox. 2.8%")
        print("  - Treasury Bond a 30 años: Rendimiento anual aprox. 3%")
        print(" Argentina:")
        print("  - Bonos soberanos/corporativos (Ej: AL30): Rendimiento anual aprox. 20%-25%")
        print("  - Plazo Fijo en Banco Santander Río (180 días): TNA aprox. 60%-70%")
    else:
        print("Horizonte no válido para recomendaciones de renta fija.")

def recomendar_etfs(horizonte):
    """
    Recomienda ETFs (SPY y QQQ) según el horizonte de inversión.

    Horizonte:
      1 → Corto plazo: Alta liquidez, aunque mayor volatilidad.
      2 → Mediano plazo: Buenas para capturar crecimiento.
      3 → Largo plazo: Históricamente sólidas para estrategias a largo plazo.
    """
    print("\n🌐 Recomendaciones de ETFs:")
    if horizonte == 1:
        print("Para un horizonte de corto plazo:")
        print(" - SPY (S&P 500): Alta liquidez y diversificación; útil en movimientos de mercado a corto plazo.")
        print(" - QQQ (Nasdaq): Ofrece exposición a tecnológicas con mayor volatilidad.")
    elif horizonte == 2:
        print("Para un horizonte de mediano plazo:")
        print(" - SPY y QQQ: Permiten capturar el crecimiento del mercado estadounidense con buena diversificación.")
    elif horizonte == 3:
        print("Para un horizonte de largo plazo:")
        print(" - SPY y QQQ: Tienen un historial probado de solidez y crecimiento, ideales para inversión a largo plazo.")
    else:
        print("Horizonte no válido para recomendaciones de ETFs.")

def main():
    # Bucle externo para repetir la sesión completa si el usuario así lo decide.
    while True:
        # Paso 1: Ingreso del capital
        while True:
            capital_input = input("\n¿Cuánto capital tenés para invertir? $: ")
            try:
                capital = float(capital_input)
                if capital > 0:
                    break
                else:
                    print("El capital debe ser mayor a 0.")
            except ValueError:
                print("Ingresá un número válido.")

        # Paso 2: Selección de parámetros (Perfil, Horizonte y Distribución)
        while True:  # Bloque de perfil y horizonte
            perfil = obtener_perfil()
            if perfil is None:
                print("⤺ Regresando al ingreso del capital...")
                break  # Se reinicia la sesión; volverá a pedir el capital
            horizonte = obtener_horizonte()
            if horizonte is None:
                print("⤺ Volviendo a la selección de perfil...")
                continue  # Repetir el bloque de perfil/horizonte
            # Bloque de Distribución
            while True:
                print("\n¿Querés usar una distribución sugerida según tu perfil o crear la tuya?")
                print("1 - Usar distribución sugerida")
                print("2 - Crear mi propia distribución")
                print("0 - Volver")
                opcion = input("Elegí una opción (0/1/2): ")
                if opcion not in ["0", "1", "2"]:
                    print("Opción inválida. Por favor, ingrese 0, 1 o 2.")
                    continue
                if opcion == "0":
                    print("⤺ Volviendo a la selección de perfil/horizonte...")
                    break  # Sale del bloque de distribución para regresar al perfil/horizonte
                elif opcion == "1":
                    distribucion = recomendar_portafolio(perfil)
                    break
                elif opcion == "2":
                    distribucion = elegir_distribucion_personalizada()
                    if distribucion is None:
                        print("⤺ Volviendo a la selección de distribución...")
                        continue  # Repetir la pregunta de distribución
                    else:
                        break
            if opcion == "0":
                continue  # Regresar al bloque de perfil/horizonte
            break  # Se obtuvieron perfil, horizonte y distribución válidos
        if perfil is None:
            continue  # Reinicia toda la sesión (se vuelve a pedir el capital)

        # Paso 3: Mostrar resultados y análisis
        mostrar_desglose(distribucion, capital)
        mostrar_grafico(distribucion)

        if distribucion.get("Acciones", 0) > 0:
            accionistas = [
                "AAPL", "MSFT", "GOOGL", "TSLA", "NVDA", "META", "AMZN",
                "AMD", "NFLX", "DIS", "INTC", "CRM", "PYPL", "BA", "NKE",
                "UBER", "BABA", "ORCL", "IBM"
            ]
            analizar_acciones_rsi_macd(accionistas, horizonte)

        if distribucion.get("Criptomonedas", 0) > 0:
            print("\n🪙 Criptomonedas sugeridas:")
            crypto_tickers = ["BTC-USD", "ETH-USD", "SOL-USD"]
            analizar_cripto(crypto_tickers, horizonte)

        if distribucion.get("Renta fija", 0) > 0:
            recomendar_renta_fija(horizonte)

        if distribucion.get("ETFs", 0) > 0:
            recomendar_etfs(horizonte)

        print("\n✅ Gracias por usar el asistente. ¡Mucho éxito con tus inversiones!")
        repetir = input("\n¿Quieres hacerlo de nuevo? (SI/NO): ").strip().lower()
                # Preguntar al final si se desea repetir la sesión.
        while True:
            repetir = input("\n¿¿Quieres hacerlo de nuevo? (SI/NO): ").strip().lower()
            if repetir in ("si", "no"):
                break
            else:
                print("Error: Debés ingresar SI o NO.")

        if repetir == "no":
            print("\nGracias por utilizar el asesor. ¡Hasta la próxima!")
            break

if __name__ == "__main__":
    main()


 -------💸📈 BIENVENIDO A TU ASESOR FINANCIERO PERSONAL EN PYTHON 📈💸------- 
